# Hitchhiker's Guide to Delta Lake (Scala)

This tutorial has been adapted for more clarity from its original counterpart [here](https://docs.delta.io/latest/quick-start.html). This notebook helps you quickly explore the main features of [Delta Lake](https://github.com/delta-io/delta). It provides code snippets that show how to read from and write to Delta Lake tables from interactive, batch, and streaming queries.

Here's what we will cover:
* Create a table
* Understanding meta-data
* Read data
* Update table data
* Overwrite table data
* Conditional update without overwrite
* Read older versions of data using Time Travel
* Write a stream of data to a table
* Read a stream of changes from a table

## Configuration
Make sure you modify this as appropriate.

In [3]:
val sessionId = scala.util.Random.nextInt(1000000)
val deltaTablePath = s"/delta/delta-table-$sessionId";

sessionId: Int = 37857
deltaTablePath: String = /delta/delta-table-37857

## Create a table
To create a Delta Lake table, write a DataFrame out in the **delta** format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

These operations create a new Delta Lake table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta Lake table, see Create a table and Write to a table (subsequent cells in this notebook).

In [4]:
val data = spark.range(0, 5)
data.show
data.write.format("delta").save(deltaTablePath)

data: org.apache.spark.sql.Dataset[Long] = [id: bigint]
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

## Understanding Meta-data

In Delta Lake, meta-data is no different from data i.e., it is stored next to the data. Therefore, an interesting side-effect here is that you can peek into meta-data using regular Spark APIs. 

In [5]:
spark.read.text(s"$deltaTablePath/_delta_log/").collect.foreach(println)

[{"commitInfo":{"timestamp":1587775682537,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true}}]
[{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}]
[{"metaData":{"id":"39cc68a2-bebd-45fd-a8d9-89b42d5b1030","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1587775677387}}]
[{"add":{"path":"part-00000-e2a83c79-4d83-4b5f-9088-7df3765c0067-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1587775681000,"dataChange":true}}]
[{"add":{"path":"part-00001-1778dccd-864e-41a8-9353-b436dbe7cb7a-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1587775681000,"dataChange":true}}]
[{"add":{"path":"part-00003-1334d226-6422-47c3-8bef-217ddc8bb84f-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":15877756810

## Read data

You read data in your Delta Lake table by specifying the path to the files: "/tmp/delta-table-scala"

In [6]:
val df = spark.read.format("delta").load(deltaTablePath)
df.show()

df: org.apache.spark.sql.DataFrame = [id: bigint]
+---+
| id|
+---+
|  4|
|  1|
|  0|
|  2|
|  3|
+---+

## Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table.


In [7]:
val data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save(deltaTablePath)
df.show()

data: org.apache.spark.sql.Dataset[Long] = [id: bigint]
+---+
| id|
+---+
|  5|
|  9|
|  6|
|  7|
|  8|
+---+

When you now inspect the meta-data, what you will notice is that the original data is over-written. Well, not in a true sense but appropriate entries are added to Delta's transaction log so it can provide an "illusion" that the original data was deleted. We can verify this by re-inspecting the meta-data. You will see several entries indicating reference removal to the original data.

In [8]:
spark.read.text(s"$deltaTablePath/_delta_log/").collect.foreach(println)

[{"commitInfo":{"timestamp":1587775707645,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"readVersion":0,"isBlindAppend":false}}]
[{"add":{"path":"part-00000-88a5a5b0-4d8e-4b48-9b67-ae1136030a8d-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1587775705000,"dataChange":true}}]
[{"add":{"path":"part-00001-82db0768-f100-48be-92f4-7bdeeae96f0c-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1587775705000,"dataChange":true}}]
[{"add":{"path":"part-00003-5effdd56-e71a-4ad9-b258-91d2f7bede50-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1587775706000,"dataChange":true}}]
[{"add":{"path":"part-00004-04aa023c-4b96-4699-aa3e-5e849124ced8-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1587775705000,"dataChange":true}}]
[{"add":{"path":"part-00006-a49fd404-72e7-4136-bfde-a108e2878f19-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1587775706

## Save as catalog tables

Delta Lake can write to managed or external catalog tables.

In [9]:
// Write data to a new managed catalog table.
data.write.format("delta").saveAsTable("ManagedDeltaTable")

In [10]:
// Define an external catalog table that points to the existing Delta Lake data in storage.
spark.sql(s"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '$deltaTablePath'")

res12: org.apache.spark.sql.DataFrame = []

In [11]:
// List the 2 new tables.
spark.sql("SHOW TABLES").show

// Explore their properties.
spark.sql("DESCRIBE EXTENDED ManagedDeltaTable").show(truncate=false)
spark.sql("DESCRIBE EXTENDED ExternalDeltaTable").show(truncate=false)

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
| default|externaldeltatable|      false|
| default| manageddeltatable|      false|
+--------+------------------+-----------+

+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                              |comment|
+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|id                          |bigint                                                                                                 |null   |
|                            |                                                                                                       |       |
|# Detailed Table Information|  

## Conditional update without overwrite

Delta Lake provides programmatic APIs to conditional update, delete, and merge (upsert) data into tables. For more information on these operations, see [Table Deletes, Updates, and Merges](https://docs.delta.io/latest/delta-update.html).

In [12]:
import io.delta.tables._
import org.apache.spark.sql.functions._

val deltaTable = DeltaTable.forPath(deltaTablePath)

import io.delta.tables._
import org.apache.spark.sql.functions._
deltaTable: io.delta.tables.DeltaTable = io.delta.tables.DeltaTable@40e3875b

In [13]:
// Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = Map("id" -> expr("id + 100")))
deltaTable.toDF.show

+---+
| id|
+---+
|  5|
|  9|
|106|
|108|
|  7|
+---+

In [14]:
// Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))
deltaTable.toDF.show

+---+
| id|
+---+
|  5|
|  9|
|  7|
+---+

In [15]:
// Upsert (merge) new data
val newData = spark.range(0, 20).toDF

deltaTable.as("oldData").
  merge(
    newData.as("newData"),
    "oldData.id = newData.id").
  whenMatched.
  update(Map("id" -> lit(-1))).
  whenNotMatched.
  insert(Map("id" -> col("newData.id"))).
  execute()

deltaTable.toDF.show()

newData: org.apache.spark.sql.DataFrame = [id: bigint]
+---+
| id|
+---+
|  1|
|  8|
| -1|
| 11|
| 18|
|  6|
| 19|
| -1|
|  2|
| 13|
|  0|
| 17|
|  4|
| 15|
| 10|
| -1|
| 12|
| 14|
|  3|
| 16|
+---+

## History
Delta's most powerful feature is the ability to allow looking into history i.e., the changes that were made to the underlying Delta Table. The cell below shows how simple it is to inspect the history.

In [16]:
deltaTable.history.show(false)

+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+
|version|timestamp          |userId|userName|operation|operationParameters                                                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|
+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+
|4      |2020-04-25 00:49:53|null  |null    |MERGE    |[predicate -> (oldData.`id` = newData.`id`)]                       |null|null    |null     |3          |null          |false        |
|3      |2020-04-25 00:49:32|null  |null    |DELETE   |[predicate -> ["((`id` % CAST(2 AS BIGINT)) = CAST(0 AS BIGINT))"]]|null|null    |null     |2          |null          |false        |
|2      |2020-04-25 00:49:15|null  |null    |UPDATE   |

## Read older versions of data using Time Travel

You can query previous snapshots of your Delta Lake table by using a feature called Time Travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.

Once you run the cell below, you should see the first set of data, from before you overwrote it. Time Travel is an extremely powerful feature that takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see [Query an older snapshot of a table (time travel)](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

In [17]:
val df = spark.read.format("delta").option("versionAsOf", 0).load(deltaTablePath)
df.show()

df: org.apache.spark.sql.DataFrame = [id: bigint]
+---+
| id|
+---+
|  4|
|  1|
|  0|
|  2|
|  3|
+---+

## Write a stream of data to a table

You can also write to a Delta Lake table using Spark's Structured Streaming. The Delta Lake transaction log guarantees exactly-once processing, even when there are other streams or batch queries running concurrently against the table. By default, streams run in append mode, which adds new records to the table.

For more information about Delta Lake integration with Structured Streaming, see [Table Streaming Reads and Writes](https://docs.delta.io/latest/delta-streaming.html).

In the cells below, here's what we are doing:

1. *Cell 23* Setup a simple Spark Structured Streaming job that generate a sequence
2. *Cell 23* Make the job write into our Delta Table
3. *Cell 25* Show the newly appended data
4. *Cell 26* Inspect history
5. *Cell 27* Stop the structured streaming job
6. *Cell 28* Inspect history <-- You'll notice appends have stopped

In [18]:
val streamingDf = spark.readStream.format("rate").load()
val stream = streamingDf.select($"value" as "id").writeStream.format("delta").option("checkpointLocation", s"/tmp/checkpoint-$sessionId").start(deltaTablePath)

streamingDf: org.apache.spark.sql.DataFrame = [timestamp: timestamp, value: bigint]
stream: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@78a5a52b

## Read a stream of changes from a table

While the stream is writing to the Delta Lake table, you can also read from that table as streaming source. For example, you can start another streaming query that prints all the changes made to the Delta Lake table.

In [19]:
deltaTable.toDF.sort($"id".desc).show

+---+
| id|
+---+
| 19|
| 18|
| 17|
| 16|
| 15|
| 14|
| 13|
| 12|
| 11|
| 10|
|  8|
|  6|
|  4|
|  3|
|  2|
|  1|
|  0|
| -1|
| -1|
| -1|
+---+

In [20]:
deltaTable.history.show

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|      5|2020-04-25 00:50:37|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          4|          null|         true|
|      4|2020-04-25 00:49:53|  null|    null|           MERGE|[predicate -> (ol...|null|    null|     null|          3|          null|        false|
|      3|2020-04-25 00:49:32|  null|    null|          DELETE|[predicate -> ["(...|null|    null|     null|          2|          null|        false|
|      2|2020-04-25 00:49:15|  null|    null|          UPDATE|[predicate -> ((i...|null|    null|     null

In [21]:
stream.stop

In [22]:
deltaTable.history.show

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|      5|2020-04-25 00:50:37|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          4|          null|         true|
|      4|2020-04-25 00:49:53|  null|    null|           MERGE|[predicate -> (ol...|null|    null|     null|          3|          null|        false|
|      3|2020-04-25 00:49:32|  null|    null|          DELETE|[predicate -> ["(...|null|    null|     null|          2|          null|        false|
|      2|2020-04-25 00:49:15|  null|    null|          UPDATE|[predicate -> ((i...|null|    null|     null

## Convert Parquet to Delta
You can do an in-place conversion from the Parquet format to Delta.## Cell title


In [23]:
val parquetPath = s"/parquet/parquet-table-$sessionId"

val data = spark.range(0,5)
data.write.parquet(parquetPath)

// Confirm that the data isn't in the Delta format
DeltaTable.isDeltaTable(parquetPath)

parquetPath: String = /parquet/parquet-table-37857
data: org.apache.spark.sql.Dataset[Long] = [id: bigint]
res41: Boolean = false

In [24]:
DeltaTable.convertToDelta(spark, s"parquet.`$parquetPath`")

// Confirm that the converted data is now in the Delta format
DeltaTable.isDeltaTable(parquetPath)

res42: io.delta.tables.DeltaTable = io.delta.tables.DeltaTable@4d80e849
res45: Boolean = true

## SQL Support
Delta supports table utility commands through SQL.  You can use SQL to:
* Get a DeltaTable's history
* Vacuum a DeltaTable
* Convert a Parquet file to Delta


In [25]:
spark.sql(s"DESCRIBE HISTORY delta.`$deltaTablePath`").show()

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+
|      5|2020-04-25 00:50:37|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          4|          null|         true|
|      4|2020-04-25 00:49:53|  null|    null|           MERGE|[predicate -> (ol...|null|    null|     null|          3|          null|        false|
|      3|2020-04-25 00:49:32|  null|    null|          DELETE|[predicate -> ["(...|null|    null|     null|          2|          null|        false|
|      2|2020-04-25 00:49:15|  null|    null|          UPDATE|[predicate -> ((i...|null|    null|     null

In [26]:
spark.sql(s"VACUUM delta.`$deltaTablePath`").show()

Deleted 0 files and directories in a total of 1 directories.
+--------------------+
|                path|
+--------------------+
|abfss://data@arca...|
+--------------------+

In [27]:
val parquetId = scala.util.Random.nextInt(1000)
val parquetPath = s"/parquet/parquet-table-$sessionId-$parquetId"

val data = spark.range(0,5)
data.write.parquet(parquetPath)

// Confirm that the data isn't in the Delta format
DeltaTable.isDeltaTable(parquetPath)

// Use SQL to convert the parquet table to Delta
spark.sql(s"CONVERT TO DELTA parquet.`$parquetPath`")

DeltaTable.isDeltaTable(parquetPath)

parquetId: Int = 228
parquetPath: String = /parquet/parquet-table-37857-228
data: org.apache.spark.sql.Dataset[Long] = [id: bigint]
res52: Boolean = false
res55: org.apache.spark.sql.DataFrame = []
res57: Boolean = true